<a href="https://colab.research.google.com/github/CHARISPRISCILLA/CODSOFT/blob/main/Titanic_Survival_Prediction_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
data = pd.read_csv("/content/tested.csv")  # Replace with the actual dataset file path
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
data.shape

(418, 12)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [ ]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
a = data.columns
#print(a[1])

for i, j in enumerate(a):
    nan_count = data[j].isna().sum().sum()

    print(f"Sum of NaN values in {a[i]} is : {nan_count}")

Sum of NaN values in PassengerId is : 0
Sum of NaN values in Survived is : 0
Sum of NaN values in Pclass is : 0
Sum of NaN values in Name is : 0
Sum of NaN values in Sex is : 0
Sum of NaN values in Age is : 86
Sum of NaN values in SibSp is : 0
Sum of NaN values in Parch is : 0
Sum of NaN values in Ticket is : 0
Sum of NaN values in Fare is : 1
Sum of NaN values in Cabin is : 327
Sum of NaN values in Embarked is : 0


In [ ]:
# Handle missing values (e.g., fill missing ages with the mean age)
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)

In [ ]:
class_survival = data.groupby(['Pclass', 'Survived']).size().unstack()
class_survival = class_survival.div(class_survival.sum(axis=1), axis=0)  # Calculate survival rate

fig = px.bar(class_survival, x=class_survival.index, y=[0, 1], labels={'0': 'Did Not Survive', '1': 'Survived'},
             title='Survival Rate by Class', barmode='group')
fig.update_xaxes(title='Class')
fig.update_yaxes(title='Survival Rate')
fig.show()

In [ ]:
fig = px.histogram(data, x='Age', color='Survived',
                   title='Age Distribution for Survivors and Non-Survivors',
                   labels={'Age': 'Age'})
fig.update_xaxes(title='Age')
fig.update_yaxes(title='Count')
fig.show()


In [ ]:
fig = px.scatter(data, x='Age', y='Fare', color='Survived', title='Scatter Plot of Age vs. Fare')
fig.update_xaxes(title='Age')
fig.update_yaxes(title='Fare')
fig.show()


In [ ]:
gender_distribution = data['Sex'].value_counts()

fig = px.pie(gender_distribution, names=gender_distribution.index, values=gender_distribution.values,
             title='Gender Distribution')
fig.show()


In [ ]:
# Encode categorical features like 'Sex' and 'Embarked'
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Embarked'] = le.fit_transform(data['Embarked'])

In [ ]:
# Select relevant features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = data[features]
y = data['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=32)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=32)

In [ ]:
y_pred = model.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("-------------------------------------------------------")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("-------------------------------------------------------")
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("-------------------------------------------------------")

Confusion Matrix:
 [[50  0]
 [ 0 34]]
-------------------------------------------------------
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

-------------------------------------------------------
Accuracy Score: 1.0
-------------------------------------------------------
